In [1]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

'''Preparing Data'''
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')
X, y = mnist["data"], mnist["target"]
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

# '''Cropping Data'''
# X_train, y_train = X_train[:1000], y_train[:1000]

Using TensorFlow backend.


In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid = [
{'n_neighbors': [3, 5, 7, 30]},
]

clf = KNeighborsClassifier(n_jobs=-1)

grid_search = GridSearchCV(clf, param_grid, cv=10)
print(grid_search.fit(X_train, y_train))

GridSearchCV(cv=10, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'n_neighbors': [1, 3, 10, 30]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)


In [12]:
cvres = grid_search.cv_results_

from pprint import pprint
pprint(grid_search.cv_results_)

import csv

with open('grid_search_details.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in grid_search.cv_results_.items():
       writer.writerow([key, value])
    
print(grid_search.best_estimator_)
np.savetxt("best_estimator.txt", [grid_search.best_estimator_],fmt="%s" )

{'mean_fit_time': array([0.01604273, 0.01549008, 0.01402569, 0.01418343]),
 'mean_score_time': array([0.10723689, 0.1068531 , 0.10674214, 0.10697496]),
 'mean_test_score': array([0.867, 0.876, 0.848, 0.78 ]),
 'mean_train_score': array([1.        , 0.92877399, 0.87789274, 0.79367635]),
 'param_n_neighbors': masked_array(data=[1, 3, 10, 30],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object),
 'params': [{'n_neighbors': 1},
            {'n_neighbors': 3},
            {'n_neighbors': 10},
            {'n_neighbors': 30}],
 'rank_test_score': array([2, 1, 3, 4], dtype=int32),
 'split0_test_score': array([0.79047619, 0.82857143, 0.79047619, 0.77142857]),
 'split0_train_score': array([1.        , 0.92960894, 0.88044693, 0.79888268]),
 'split1_test_score': array([0.88349515, 0.87378641, 0.83495146, 0.75728155]),
 'split1_train_score': array([1.        , 0.92307692, 0.87736901, 0.79487179]),
 'split2_test_score': array([0.91176471, 0.93137255, 0.8

In [13]:
final_model = grid_search.best_estimator_
final_predictions = final_model.predict(X_test)

In [18]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
conf_mx = confusion_matrix(y_test, final_predictions)
print(accuracy_score(y_test, final_predictions))
print(conf_mx)
np.savetxt("accuracy_score.txt", [accuracy_score(y_test, final_predictions)], delimiter="," , fmt='%10.5f')
np.savetxt("conf_mat.csv", conf_mx, delimiter="," , fmt='%10.0f')

data = conf_mx

image_product = 70
new_data = np.zeros(np.array(data.shape) * image_product)
for j in range(data.shape[0]):
    for k in range(data.shape[1]):
        new_data[j * image_product: (j+1) * image_product, k * image_product: (k+1) * image_product] = data[j, k]
# plt.imshow(new_data, cmap=plt.cm.gray)
# plt.show()
plt.imsave("confusion_mat.jpg" , new_data, dpi=1000,cmap=plt.cm.gray)

0.8755
[[ 957    2    4    2    0    4    8    1    2    0]
 [   0 1130    0    4    0    0    1    0    0    0]
 [  31   82  845   16    3    1    9   33    9    3]
 [   4   30   15  916    0   20    3   11    8    3]
 [   4   38    5    2  802    0    9    7    1  114]
 [  17   27    3  114    7  679   18    5    7   15]
 [  11   13    4    2    6   10  910    1    1    0]
 [   0   62    5    1   12    0    0  911    0   37]
 [  28   24   28  103   12   28    3   11  709   28]
 [  13   16    2   10   46    5    1   20    0  896]]


In [ ]:
# from sklearn.model_selection import cross_val_predict,cross_val_score
# clf = KNeighborsClassifier(n_jobs=-1)
# y_train_pred = cross_val_score(clf, X_train, y_train, cv=10)
# print(y_train_pred)
# y_train_pred = cross_val_predict(clf, X_train, y_train, cv=10)
# print(y_train_pred)
